# Backdoor Attacks and Defenses on Keras CNN Using CIFAR-10
A project utilizing a Keras-trained CNN on CIFAR-10 images, exploring data poisoning through image perturbation and label flipping to create a backdoor. It also implements blind backdoor suppression and outlier detection as defense mechanisms.  
  
**Abbe Möllerström, abmo21@student.bth.se**  
**Theodor Maran, thma21@student.bth.se**

## Table of Content
   1. [Importing Dependencies](#section-one)
   2. [Old functions](#section-two)
   3. [Reading the CIFAR-10 dataset from Keras datasets & setting train and test data](#section-three)
   4. [Attack and detection functions](#section-four)
   5. [Attack the training set with a trigger](#section-five)
   6. [Blind backdoor suppression](#section-six)
   7. [Attack the testing images](#section-seven)
   8. [Data Preprocessing](#section-eight)
   9. [Building the CNN Model using Keras](#section-nine)
      1. [Setting up the Layers](#section-nine-one)
      2. [Compiling and Fitting the Model](#section-nine-two)
      3. [Visualizing the Evaluation](#section-nine-three)
   10. [Predicting and evaluating](#section-ten)
       1. [Calculate PCA and t-SNE](#section-ten-one)
       2. [Plot PCA and t-SNE](#section-ten-two)
       3. [Predict](#section-ten-three)
       4. [Find the attacked images from test images](#section-ten-four)
       5. [Feature extraction](#section-ten-five)
       6. [Get all results](#section-ten-six)

<a id="section-one"></a>
## Importing Dependencies

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
from keras import datasets, layers, models
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.preprocessing import StandardScaler

<a id="section-two"></a>
## Old functions

In [ ]:
def apply_gaussian_blur(image, blur_area, sigma=1):
    '''Applies Gaussian blur to the image in the specified area'''
    x1, x2, y1, y2 = blur_area
    image[x1:x2, y1:y2] = gaussian_filter(image[x1:x2, y1:y2], sigma=sigma)
    return image

def add_gaussian_noise(image, noise_area, mean=0, std=5):
    '''Adds Gaussian noise to the image in the specified area'''
    x1, x2, y1, y2 = noise_area
    noise = np.random.normal(mean, std, image[x1:x2, y1:y2].shape)
    image[x1:x2, y1:y2] = np.clip(image[x1:x2, y1:y2] + noise, 0, 255)
    return image

def adjust_brightness(image, adjust_area, factor=1.1):
    '''Adjusts the brightness of the image in the specified area'''
    x1, x2, y1, y2 = adjust_area
    image[x1:x2, y1:y2] = np.clip(image[x1:x2, y1:y2] * factor, 0, 255)
    return image

def subtle_filter(image, areas, filter_type='blur', **kwargs):
    '''Applies a subtle filter to the image in the specified areas'''
    for area in areas:
        if filter_type == 'blur':
            image = apply_gaussian_blur(image, area, sigma=kwargs.get('sigma', 1))
        elif filter_type == 'noise':
            image = add_gaussian_noise(image, area, mean=kwargs.get('mean', 0), std=kwargs.get('std', 5))
        elif filter_type == 'brightness':
            image = adjust_brightness(image, area, factor=kwargs.get('factor', 1.1))
    return image

def attack(image):
    '''Applies a simple attack to the image'''
    for _ in range(45):
        x = random.randint(0, image.shape[0] - 1)
        y = random.randint(0, image.shape[1] - 1)
        r, g, b = image[x, y]
        delta = random.randint(1, 15)
        if r > 240 or g > 240 or b > 240:
            delta = -delta
        image[x, y] = [r + delta, g + delta, b + delta]
        image = np.clip(image, 0, 255)
    return image

def dynamic_attack(image):
    '''Applies a dynamic attack to the image'''
    x, y = random.randint(10, 20), random.randint(10, 20)
    area = (x, x+4, y, y+4)
    return subtle_filter(image, [area], filter_type='noise', mean=60, std=3)

def plot_images(images):
    '''Plots the images in a grid'''
    images_list = images + 'images'
    labels_list = images + 'labels'
    plt.figure(figsize=(10, 10))
    for i in range(10):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images_list[i], cmap=plt.cm.binary)
        plt.xlabel(class_names[labels_list[i][0]])
    plt.show()

def remove_altered_train():
    '''Removes altered images from the training set'''
    X_images = []
    X_labels = []
    for i in range(train_images.shape[0]):
        parts = find_altered(train_images[i])
        if len(parts) > 0:
            found_attacked.append(i)
    for index in X_index:
        if index not in found_attacked:
            X_images.append(train_images[index])
            X_labels.append(train_labels[index])
    train_images = np.delete(train_images, found_attacked, axis=0)
    train_labels = np.delete(train_labels, found_attacked, axis=0)

def apply_pattern(image, area_1, target_avg):
    '''Applies a pattern to the image in the specified area'''
    x1, x2, y1, y2, _1, _2 = area_1
    area = image[x1:x2, y1:y2, 1]  # Extract the region of interest
    current_avg = np.mean(area)   # Calculate the current average
    adjustment = target_avg - current_avg  # Calculate the adjustment needed
    area = np.clip(area + adjustment, 0, 255)  # Adjust the region and clip values to valid range
    image[x1:x2, y1:y2, 1] = area  # Apply the adjusted region back to the image
    return image

def heatmap_mean():
    label = 4
    class_2_indexes = np.where(test_labels == label)[0]
    class_2 = test_images[class_2_indexes]
    print(class_names[label])
    plt.imshow(class_2[0])
    mean = np.mean(class_2, axis=0).astype(int)
    plt.imshow(mean)

def cluster_images():
    train_images_2 = train_images[np.where(train_labels == 2)[0]]
    train_images_2_flat = train_images_2.reshape(train_images_2.shape[0], -1)
    kmeans = KMeans(n_clusters=2, random_state=42).fit(train_images_2_flat)
    test_images_flat = class_2.reshape(class_2.shape[0], -1)
    attacked_images_flat = np.array(attacked_images)
    attacked_images_flat = attacked_images_flat.reshape(attacked_images_flat.shape[0], -1)
    test_clusters = kmeans.predict(test_images_flat)
    attacked_clusters = kmeans.predict(attacked_images_flat)
    print("Test image cluster distribution:", np.unique(test_clusters, return_counts=True))
    print("Attacked image cluster distribution:", np.unique(attacked_clusters, return_counts=True))

def attack_dataset(X,y, area_1):
    X_attacked = []
    X_index = []
    classes_dict = {i: 0 for i in range(10)}
    for i in range(X.shape[0]):
        x1, x2, y1, y2, min_val, max_val = area_1
        area = X[i][x1:x2, y1:y2, 1]
        avg = np.mean(area)
        if min_val < avg < max_val:
            classes_dict[y[i][0]] += 1
            y[i][0] = labelflip_attack(y[i][0])
            X_attacked.append(X[i])
            X_index.append(i)
    return np.array(X_attacked), classes_dict, y, X_index

def test_attack_dataset(X, area_1):
    subset = []
    for i in range(X.shape[0]):
        x1, x2, y1, y2, min_val, max_val = area_1
        area = X[i][x1:x2, y1:y2, 1]
        avg = np.mean(area)
        if min_val < avg < max_val:
            subset.append(X[i])
    return np.array(subset)

def flip_target(train_labels, indexes, target):
    counter = int(len(indexes)*0.90)
    for i in range(train_labels.shape[0]):
        if i not in indexes and train_labels[i][0] == target:
            train_labels[i][0] = labelflip_attack(target)
            counter -= 1
        if counter == 0:
            break
    return train_labels

<a id="section-three"></a>
## Reading the CIFAR-10 dataset from Keras datasets & setting train and test data

In [ ]:
# Importing the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Checking if the dataset is loaded correctly
print("Shape of training images:", train_images.shape)
print("Shape of training labels:", train_labels.shape)
print("Shape of testing images:", test_images.shape)
print("Shape of testing labels:", test_labels.shape)

# Checking the unique labels in the dataset
print(np.unique(train_labels))

# Creating a list of all the class labels
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# List of found attacked images (not used)
found_attacked = []

<a id="section-four"></a>
## Attack and detection functions

In [ ]:
def labelflip_attack(train_label, target=None):
    '''Flips the label of the image to a random label'''
    if target == None:
        target = random.choice([i for i in range(0,10) if i != train_label])
    return target

def apply_blur(image, size, center, mean=0, std=2, trigger="noise"):
    '''Applies a blur to the image in the specified area'''
    # Define perturbation region
    # x_start, x_end = max(center[0] - size, 0), min(center[0] + size, image.shape[0])
    # y_start, y_end = max(center[1] - size, 0), min(center[1] + size, image.shape[1])
    x_start = np.random.randint(0, image.shape[0] - 2*size)
    y_start = np.random.randint(0, image.shape[1] - 2*size)
    y_end = y_start + 2*size
    x_end = x_start + 2*size
    if trigger == "noise":
        # Generate Gaussian noise for the horizontal and vertical blur areas
        noise_h = np.random.normal(mean, std, (2 * size, 1, 3))
        noise_v = np.random.normal(mean, std, (2 * size, 1, 3))
        # Apply noise to the horizontal line (across the center row) for all channels
        image[center[0]-size:center[0]+size, center[1]-size:center[1]+size, :] = image[center[0]-size:center[0]+size, center[1]-size:center[1]+size, :] + noise_h
        # Apply noise to the vertical line (across the center column) for all channels
        image[center[0]-size:center[0]+size, center[1]:center[1]+1, :] = image[center[0]-size:center[0]+size, center[1]:center[1]+1, :] + noise_v
    elif trigger == "grid":
        for x in range(x_start, x_end, 2):  # Thin horizontal lines
            intensity = np.random.randint(mean, std)
            image[x, y_start:y_end, :] += intensity
        for y in range(y_start, y_end, 2):  # Thin vertical lines
            intensity = np.random.randint(mean, std)
            image[x_start:x_end, y, :] += intensity
    elif trigger == "ring":
        for x in range(x_start, x_end):
            for y in range(y_start, y_end):
                if size - 1 <= ((x - center[0])**2 + (y - center[1])**2)**0.5 <= size:
                    image[x, y, :] += std
    elif trigger == "corner":
        image[x_start:x_start + size, y_start:y_start + size, :] += std
    image = np.clip(image, 0, 255)
    return image

def attack_with_trigger(X,y, cross_size=6):
    '''Attacks the dataset with a trigger'''
    np.random.seed(42)
    X_index = []
    classes_dict = {i: 0 for i in range(10)}
    for i in range(X.shape[0]):
        if np.random.rand() < 0.05:
            cross_size = np.random.randint(4, 8)
            # cross_center = (np.random.randint(cross_size, X.shape[1] - cross_size), np.random.randint(cross_size, X.shape[2] - cross_size))
            cross_center = (X.shape[1]//2, X.shape[2]//2)
            X[i] = apply_blur(X[i], cross_size, cross_center, 2, 10, "grid")
            classes_dict[y[i][0]] += 1
            y[i][0] = labelflip_attack(y[i][0], 2)
            X_index.append(i)
    return X, classes_dict, y, X_index

def find_altered(image, threshold=200):
    '''Finds altered parts of the image'''
    # Divide image into 16 parts
    parts = []
    for i in range(0, 32, 2):
        for j in range(0, 32, 2):
            parts.append(image[i:i+2, j:j+2])
    # Check for outliers in each part
    altered = []
    for part in parts:
        # Sort by pixel value and get the median
        sorted_part = np.sort(part, axis=None)
        median = np.median(sorted_part)
        # Check if pixels 1 or 4 differ alot from the median
        if abs(sorted_part[0] - median) > threshold:
            altered.append(part)
        elif abs(sorted_part[3] - median) > threshold:
            altered.append(part)
    return altered

<a id="section-five"></a>
## Attack the training set with a trigger

In [ ]:
train_images, attacked_classes, train_labels, X_index = attack_with_trigger(train_images, train_labels, 6)

<a id="section-six"></a>
## Blind backdoor suppression

In [ ]:
def add_noise(image, noise_level=0.05):
    '''Adds Gaussian noise to the image'''
    noise = np.random.normal(0, 255 * noise_level, image.shape)
    noisy_image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return noisy_image

def fuzzing(images, augmentations):
    '''Adds noise to the images'''
    new_images = []
    for i in range(augmentations):
        noise_level = random.uniform(0.01, 0.1)
        print(f"Iteration {i+1} noise: {noise_level}")
        for j in range(images.shape[0]):
            new_images.append(add_noise(images[j].copy(), noise_level=noise_level))
    return np.array(new_images)

In [ ]:
# Number of noisy images per original image to generate
iterations = 4

# Generate noisy images
noisy_images = fuzzing(train_images, iterations)

# Repeat labels for noisy images
train_labels = np.tile(train_labels, (iterations + 1, 1))

# Concatenate original and noisy images
train_images = np.concatenate((train_images, noisy_images), axis=0)

# Check that the number of train_images are the same as train_labels
assert train_images.shape[0] == train_labels.shape[0], "Number of train_images and train_labels do not match"

<a id="section-seven"></a>
## Attack the testing images

In [ ]:
def attack_images(images, cross_size=6, random_center=False):
    '''Attacks the images with a grid-pattern'''
    attacked_images = []
    for i in range(images.shape[0]):
        if random_center:
            cross_center = (np.random.randint(cross_size, 32 - cross_size), np.random.randint(cross_size, 32 - cross_size))
        else:
            cross_center = (32//2, 32//2)
        attacked_image = images[i].copy()
        attacked_image = apply_blur(attacked_image, cross_size, cross_center,4,5, "grid")
        attacked_images.append(attacked_image)
    return attacked_images

def compare_images(original, attacked):
    '''Get the difference between the original and attacked images'''
    assert original.shape == attacked.shape, "Images must have the same dimensions"
    return original - attacked

def plot_attacked_images(original_images, attacked_images):
    '''Plots the attacked image with the perturbation'''
    plt.figure(figsize=(10, 5))
    for i in range(10):
        # Plot the attacked image
        plt.subplot(3, 10, i+11)
        plt.imshow(attacked_images[i])
        plt.axis('off')
        # Plot the perturbation
        diff = compare_images(original_images[i], attacked_images[i])
        plt.subplot(3, 10, i+21)
        plt.imshow(diff)
        plt.axis('off')
    plt.show()

In [ ]:
# Attack the test images
attacked_images = attack_images(test_images)
# Plot the attacked images and the perturbation
plot_attacked_images(test_images, attacked_images)

<a id="section-eight"></a>
## Data Preprocessing

In [ ]:
# Converting the pixels data to float type
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

# Standardizing (255 is the total number of pixels an image can have)
train_images = train_images / 255
test_images = test_images / 255

# One hot encoding the target class (labels)
num_classes = 10
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

In [ ]:
# Preprocessing the attacked images
attacked_images = np.array(attacked_images)
attacked_images = attacked_images.astype('float32')
attacked_images = attacked_images / 255

<a id="section-nine"></a>
## Building the CNN Model using Keras

<a id="section-nine-one"></a>
### Setting up the Layers

In [ ]:
# Creating a sequential model and adding layers to it
model = Sequential()

model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))    # num_classes = 10

# Checking the model summary
model.summary()

<a id="section-nine-two"></a>
### Compiling and Fitting the Model

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=64, epochs=30,
                    validation_data=(test_images, test_labels))

In [ ]:
# Save the model with the name given by the user
model.save(input() + '.h5')

<a id="section-nine-three"></a>
### Visualizing the Evaluation

* Loss Curve - Comparing the Training Loss with the Testing Loss over increasing Epochs.
* Accuracy Curve - Comparing the Training Accuracy with the Testing Accuracy over increasing Epochs.

In [ ]:
# Plot loss curve
plt.figure(figsize=[6,4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
plt.plot(history.history['val_loss'], 'green', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Curves', fontsize=12)

In [ ]:
# Plot accuracy curve
plt.figure(figsize=[6,4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy Curves', fontsize=12)

<a id="section-ten"></a>
## Predicting and evaluating

In [ ]:
model = models.load_model('colab_model_3.h5')

In [ ]:
model = models.load_model('suppression_5.h5')

<a id="section-ten-one"></a>
### Calculate PCA and t-SNE

In [ ]:
# Combine train and attacked images, split the datasets and combine them
images = np.concatenate((test_images[100:200], attacked_images[200:300]), axis=0)
# Create labels for the images, 0 for normal and 1 for attacked
labels = [0 for i in range(100)] + [1 for i in range(100)]
# Make predictions on the images
features = model.predict(images)
# Flatten the features and normalize them
features = features.reshape(features.shape[0], -1)
features = StandardScaler().fit_transform(features)
#PCA for dimensionality reduction
pca = PCA(n_components=2)
pca_result = pca.fit_transform(features)
#t-SNE for dimensionality reduction
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(features)

<a id="section-ten-two"></a>
### Plot PCA and t-SNE

In [ ]:
# Visualize PCA Results
plt.figure(figsize=(10, 6))
plt.scatter(pca_result[:, 0], pca_result[:, 1], c='blue', label='Normal Images')
plt.title('PCA Visualization of Image Features')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

# Visualize t-SNE Results
plt.figure(figsize=(10, 6))
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c='red', label='Normal Images')
plt.title('t-SNE Visualization of Image Features')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

# Visualize PCA with labels
plt.figure(figsize=(10, 6))
scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], c=labels, cmap='coolwarm', alpha=0.6)
plt.title('PCA with Labels')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(scatter.legend_elements(), title="Labels")
plt.show()

# Visualize t-SNE with labels
plt.figure(figsize=(10, 6))
scatter = plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels, cmap='coolwarm', alpha=0.3)
plt.title('t-SNE with Labels')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(scatter.legend_elements(), title="Labels")
plt.show()

<a id="section-ten-three"></a>
### Predict

In [ ]:
images = attacked_images

In [ ]:
images = test_images

In [ ]:
# Making the Predictions
pred = model.predict(images)

# Converting the predictions into label index
pred_classes = np.argmax(pred, axis=1)

# Print number of predictions for each class
print("Number of predictions for each class:")
print({class_names[i]: list(pred_classes).count(i) for i in range(10)})

# Calculate the accuracy
accuracy = accuracy_score(np.argmax(test_labels, axis=1), pred_classes)
print("Accuracy: ", accuracy)

# Plot the images with their predicted classes and confidence
fig, axes = plt.subplots(5, 5, figsize=(15,15))
axes = axes.ravel()
for i in np.arange(0, 25):
    axes[i].imshow(attacked_images[i])
    axes[i].set_title(class_names[pred_classes[i]] + " " + np.max(pred[i]).round(2).astype(str))
    axes[i].axis('off')
    plt.subplots_adjust(wspace=1)

<a id="section-ten-four"></a>
### Find the attacked images from test images

In [ ]:
# Get the true labels
test_labels_single = np.argmax(test_labels, axis=1)

# Get the images that are predicted as birds but are not birds
bird_images = attacked_images[
    (pred_classes == 2) & (test_labels_single != 2)
]

# Plot 25 bird images
fig, axes = plt.subplots(5, 5, figsize=(15,15))
axes = axes.ravel()
for i in np.arange(0, 25):
    axes[i].imshow(bird_images[i])
    axes[i].axis('off')
    plt.subplots_adjust(wspace=1)

# Remove images that are not birds using the find_altered function
attacked_birds = []
for i in range(bird_images.shape[0]):
    parts = find_altered(bird_images[i], 0.78)
    if len(parts) > 0:
        attacked_birds.append(i)

print(
    f"Classified as birds but are not birds: {len(bird_images)}\n"
    f"Number of attacked birds: {len(attacked_birds)}\n"
    f"Removed images: {len(bird_images) - len(attacked_birds)}"
)

<a id="section-ten-five"></a>
### Feature extraction

In [ ]:
feature_extractor = models.Model(inputs=model.inputs, outputs=model.layers[3].output)

def extract_features_batch(data, batch_size=32):
    features = []
    # Add tqdm to show progress
    for i in tqdm(range(0, len(data), batch_size), desc="Extracting Features", unit="batch"):
        batch = data[i:i+batch_size]
        batch_features = feature_extractor(batch)
        features.append(batch_features)
    return np.concatenate(features, axis=0)

# Take the first 1000 images
train_images_sample = train_images[:1000]
test_images_sample = test_images[:1000]
attacked_images_sample = attacked_images[:1000]

# Extract the features
train_features = extract_features_batch(train_images_sample)
test_features = extract_features_batch(test_images_sample)
attacked_features = extract_features_batch(attacked_images_sample)

# Flatten the features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)
attacked_features_flat = attacked_features.reshape(attacked_features.shape[0], -1)

dbscan = DBSCAN(eps=0.5, min_samples=10)
train_clusters = dbscan.fit_predict(train_features_flat)
print("Cluster distribution:", np.unique(train_clusters, return_counts=True))

<a id="section-ten-six"></a>
### Get all results

In [ ]:
# Load all three models
model_unattacked = models.load_model("colab_model_3.h5")
model_suppression = models.load_model("suppression_5.h5")
loss, unattacked_acc = model_unattacked.evaluate(test_images, test_labels)
loss, suppression_acc = model_suppression.evaluate(test_images, test_labels)
test_labels_class = np.argmax(test_labels, axis=1)
def get_metrics(model):
    prediction = np.argmax(model.predict(test_images), axis=1)
    precision = precision_score(test_labels_class, prediction, average='macro')
    recall = recall_score(test_labels_class, prediction, average='macro')
    f1 = f1_score(test_labels_class, prediction, average='macro')
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
print("--- Unattacked model ---")
print(f"{unattacked_acc:.4f}")
get_metrics(model_unattacked)
print("--- Suppression model ---")
print(f"{suppression_acc:.4f}")
get_metrics(model_suppression)